In [65]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

def preprocess(x):
    x = str(x) #轉str
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 去除換行符號
    return str.strip(x)

In [66]:
df = pd.read_excel('../data\combined_excel.xlsx').dropna(subset=['59'],axis=0)
df['59'] = df['59'].apply(lambda x:preprocess(x).upper())
for i in df[['59']].values[:5]:
    print(i)

['FORMOSA PLASTICS CORPORATION201TUNG HWA NORTH ROADTAIPEI TAIWAN']
['FORMOSA PLASTICS CORPORATION201 TUNG HWA NORTH ROADTAIPEI TAIWAN']
['FORMOSA PLASTICS CORPORATION201 TUNG HWA N ROAD TAIPEITAIWAN']
['FORMOSA PLASTICS CORPORATION201TUNG HWA N ROADTAIPEI TAIWAN']
['NAN YA PLASTICS CORPORATION201 TUN HWA NORTH ROAD TAIPEITAIWAN 105 REPUBLIC OF CHINATEL886227178272 FAX886227178534']


In [67]:
寶典 = pd.read_excel('../data\寶典\台塑出口公司名稱及統編.xlsx')
寶典['公司英文名稱'] = 寶典['公司英文名稱'].apply(lambda x:preprocess(x))
寶典['尾綴'] = 寶典['公司英文名稱'].apply(lambda x:x.split(' ')[-1])
display(寶典)
寶典.to_csv('../data/寶典/公司寶典加尾綴.csv')

,代號,公司簡稱,公司統編,公司英文名稱,尾綴
0,1,台塑塑膠,75708007,FORMOSA PLASTICS CORPORATION,CORPORATION
1,2,南亞塑膠,75370905,NAN YA PLASTICS CORPORATION,CORPORATION
2,4,台灣化學纖維,58650902,FORMOSA CHEMICALS AND FIBRE CORP,CORP
3,6,台塑石化,86522210,FORMOSA PETROCHEMICAL CORP,CORP
4,AF,台朔汽車,16745630,FORMOSA AUTOMOBILE SALES CORPORATION,CORPORATION
5,AH,台塑旭,16446128,FORMOSA ASAHI SPANDEX CO LTD,LTD
6,AL,台塑大金,70467423,FORMOSA DAIKIN ADVANCED CHEMICALS CO LTD,LTD
7,BM,台塑生醫,80539426,FORMOSA BIOMEDICAL TECHNOLOGY CORPORATION,CORPORATION
8,BN,虹京資源,27992242,HONG JING RESOURCES CO LTD,LTD
9,GF,長庚醫學科技,24338670,CHANG GUNG MEDICAL TECHNOLOGY CO LTD,LTD


In [68]:
df = df.rename(columns={'59':'string_X_train'})
df = df[['string_X_train','EXPNO','from']].dropna(subset=['string_X_train'],axis=0)
print(df.shape)
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:]))
df['Y_label'] = 'not find'
for idx in tqdm(df.index):
    for name in 寶典['公司英文名稱'].values.tolist():# 先嘗試完全匹配寶典
        if name in df.loc[idx,'string_X_train']:
            df.loc[idx,'Y_label'] = name
            df.loc[idx,'string_X_train'] = name + ' ' + df.loc[idx,'string_X_train'].replace(name,'') # y + 空格 + 後半段
    for name in 寶典['尾綴'].values.tolist():# 匹配不到則抓尾綴
        x = df.loc[idx,'string_X_train']
        if (name in x) & (df.loc[idx,'Y_label'] == 'not find'):
            df.loc[idx,'Y_label'] = x[:x.find(name)+len(name)]
            df.loc[idx,'straing_X_train'] = x[:x.find(name)+len(name)] + ' ' + df.loc[idx,'string_X_train'].replace(x[:x.find(name)+len(name)],'') # y + 空格 + 後半段
df = df[df['Y_label'] != 'not find'] # 取成功標註的
print(df.shape)
df.head()

(20498, 3)


C:\Users\ricky\AppData\Local\Temp/ipykernel_3468/903357763.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(df.index):



(14376, 5)


,string_X_train,EXPNO,from,Y_label,straing_X_train
0,FORMOSA PLASTICS CORPORATION 201TUNG HWA NORTH...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN
1,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN
2,FORMOSA PLASTICS CORPORATION 201 TUNG HWA N RO...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN
3,FORMOSA PLASTICS CORPORATION 201TUNG HWA N ROA...,11,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN
4,NAN YA PLASTICS CORPORATION 201 TUN HWA NORTH ...,27,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN


In [69]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye
ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys),ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(df.shape)
df.head(10)

(14376, 7)


,string_X_train,EXPNO,from,Y_label,straing_X_train,string_Y_1,string_Y_2
0,FORMOSA PLASTICS CORPORATION 201TUNG HWA NORTH...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
1,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
2,FORMOSA PLASTICS CORPORATION 201 TUNG HWA N RO...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
3,FORMOSA PLASTICS CORPORATION 201TUNG HWA N ROA...,11,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
4,NAN YA PLASTICS CORPORATION 201 TUN HWA NORTH ...,27,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN,0,27
5,FORMOSA PLASTICS CORPORATION 201TUNG HWA NORTH...,18,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
6,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
7,NAN YA PLASTICS CORPORATION NO201TUNG HWA NROA...,25,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN,0,27
8,NAN YA PLASTICS CORPORATION NO201TUNG HWA NROA...,25,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN,0,27
9,NAN YA PLASTICS CORPORATION NO201TUNG HWA NROA...,25,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN,0,27


In [70]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a==b

C:\Users\ricky\AppData\Local\Temp/ipykernel_3468/2657461343.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [71]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
 NAN YA PLASTICS CORPORATION  
NAN YA PLASTICS CORPORATION
1
 FORMOSA CHEMICALS AND FIBRE CORP  201 TUNG HWA N ROAD TAIPEITAIWAN
FORMOSA CHEMICALS AND FIBRE CORP
2
 NAN YA PLASTICS CORPORATION  201 TUNG HWA N ROADTAIPEI TAIWAN ROC
NAN YA PLASTICS CORPORATION
3
 FORMOSA CHEMICALS AND FIBRE CORP  ADD201TUNG HWA N ROADTAIPEITAIWAN
FORMOSA CHEMICALS AND FIBRE CORP
4
 FORMOSA CHEMICALS AND FIBRE CORP  ORATION 201 TUNG HWA NROADTAIPEITAIWANTELEPHONE NO886227122211
FORMOSA CHEMICALS AND FIBRE CORP
5
 NAN YA PLASTICS CORPORATION  201 TUN HUA NORTH ROAD6TH FLOORTAIPEITAIWAN
NAN YA PLASTICS CORPORATION
6
 FORMOSA PLASTICS CORPORATION  201TUNG HWA NORTH ROADTAIPEITAIWAN
FORMOSA PLASTICS CORPORATION
7
 FORMOSA PETROCHEMICAL CORP  ORATIONRM 510 5F 201 TUNG HWA N RDTAIPEI TAIWAN
FORMOSA PETROCHEMICAL CORP
8
 FORMOSA CHEMICALS AND FIBRE CORP  
FORMOSA CHEMICALS AND FIBRE CORP
9
 NAN YA PLASTICS CORPORATION  201 TUN HWA TAIPEITAIWAN 105 ROC
NAN YA PLASTICS CORPORATION


In [72]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_公司.csv')
df

(14376, 7)


,string_X_train,EXPNO,from,Y_label,straing_X_train,string_Y_1,string_Y_2
0,FORMOSA PLASTICS CORPORATION 201TUNG HWA NORTH...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
1,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
2,FORMOSA PLASTICS CORPORATION 201 TUNG HWA N RO...,NaN,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
3,FORMOSA PLASTICS CORPORATION 201TUNG HWA N ROA...,11,20210103.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
4,NAN YA PLASTICS CORPORATION 201 TUN HWA NORTH ...,27,20210103.xlsx,NAN YA PLASTICS CORPORATION,NaN,0,27
...,...,...,...,...,...,...,...
20732,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,NaN,20200712.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
20733,FORMOSA PLASTICS CORPORATION 201 TUNG HWA NORT...,11,20200712.xlsx,FORMOSA PLASTICS CORPORATION,NaN,0,28
20734,NAN YA PLASTICS CORP201 TUNG HWA N ROADTAIPEI ...,NaN,20200712.xlsx,NAN YA PLASTICS CORP,NAN YA PLASTICS CORP 201 TUNG HWA N ROADTAIPEI...,0,20
20735,NAN YA PLASTICS CORP201 TUNG HWA N ROADTAIPEI ...,NaN,20200712.xlsx,NAN YA PLASTICS CORP,NAN YA PLASTICS CORP 201 TUNG HWA N ROADTAIPEI...,0,20
